In [1]:
!pip install opencv-python-headless mediapipe

In [ ]:
!python.exe -m pip install --upgrade pip

/bin/bash: line 1: python.exe: command not found


In [2]:
import cv2
import mediapipe as mp
# from google.colab.patches import cv2_imshow # For displaying images in Colab

# --- 1. Initialize MediaPipe Pose ---
# This sets up the model with its default parameters.
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# --- 2. Setup Video File ---
video_input_path = 'C:/Users/sahil/R23EQ092/Sit_Ups_3.mp4'
video_output_path = 'output_video.mp4'

# Open the video file for reading
cap = cv2.VideoCapture(video_input_path)

# Get video properties (width, height, fps) for the output file
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_output_path, fourcc, fps, (frame_width, frame_height))

print("Processing video... this may take a moment.")

# --- 3. Process Each Frame in the Video ---
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    if not success:
        break # Break the loop if we've reached the end of the video

    # MediaPipe requires RGB images, but OpenCV reads in BGR. So, we convert.
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with the MediaPipe Pose model
    results = pose.process(frame_rgb)

    # Draw the pose landmarks (the skeleton) on the original frame
    # We draw on the original 'frame' (BGR) not 'frame_rgb'
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )

    # Write the frame with the drawn landmarks to the output video file
    out.write(frame)

# --- 4. Release Resources ---
print(f"Finished processing. Output video saved to {video_output_path}")
cap.release()
out.release()
pose.close()

# Optional: Display the first frame with landmarks to verify in Colab
# cap = cv2.VideoCapture(video_output_path)
# success, first_frame = cap.read()
# if success:
#    print("Displaying the first frame of the output video:")
#    cv2_imshow(first_frame)
# cap.release()

Processing video... this may take a moment.
Finished processing. Output video saved to output_video.mp4


In [4]:
import cv2
import mediapipe as mp
import numpy as np

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0: angle = 360 - angle
    return angle

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

video_input_path = 'C:/Users/sahil/Downloads/output_video.mp4'
video_output_path = 'output_video_COUNTER.mp4'
cap = cv2.VideoCapture(video_input_path)
frame_width, frame_height = int(cap.get(3)), int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_output_path, fourcc, fps, (frame_width, frame_height))

counter = 0
stage = None

print("Starting video processing with data-driven thresholds...")

while cap.isOpened():
    success, frame = cap.read()
    if not success: break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        try:
            landmarks = results.pose_landmarks.landmark
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            angle = calculate_angle(hip, knee, ankle)

            # --- FINAL COUNTER LOGIC - CALIBRATED WITH YOUR DATA ---
            # [cite_start]This logic detects the angle oscillation to count full cycles [cite: 102]

            # Repetition starts when leg is extended (larger angle)
            if angle > 40: # Based on your observed max of ~44 degrees
                stage = "extended"

            # Repetition is counted when leg is bent (smaller angle)
            if angle < 35 and stage =='extended': # Based on your observed min of ~32 degrees
                stage="bent"
                counter +=1
                print(f"Sit-up count: {counter}")

        except: pass

    # --- Render Counter ---
    cv2.rectangle(frame, (0,0), (225,73), (245,117,16), -1)
    cv2.putText(frame, 'REPS', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(frame, str(counter), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    out.write(frame)

print(f"Final processing complete. Final sit-up count: {counter}")
cap.release()
out.release()
pose.close()

Starting video processing with data-driven thresholds...
Sit-up count: 1
Sit-up count: 2
Sit-up count: 3
Final processing complete. Final sit-up count: 3


In [1]:
!pip install --force-reinstall tensorflow==2.15.0 tensorflow-hub

  Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl.metadata (3.6 kB)
  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached tensorflow_intel-2.15.0-cp310-cp310-win_amd64.whl.metadata (5.1 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.14.0-cp310-cp310-win_amd64.whl.metadata (2.7 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-win_amd64.whl.metadata (20 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3

  You can safely remove it manually.
  You can safely remove it manually.


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import os

# Download the MoveNet model from TensorFlow Hub
model_url = "https://tfhub.dev/google/movenet/singlepose/lightning/4"
hub_model = hub.load(model_url)

# Define a function that uses the model
def movenet(input_image):
    return hub_model.signatures['serving_default'](input_image)

# Save the model in TensorFlow's SavedModel format
saved_model_dir = "movenet_saved_model"
tf.saved_model.save(hub_model, saved_model_dir)

print(f"MoveNet model downloaded and saved to '{saved_model_dir}'")

C:\Users\sahil\anaconda3\envs\SIH_1\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


INFO:tensorflow:Assets written to: movenet_saved_model\assets


INFO:tensorflow:Assets written to: movenet_saved_model\assets


MoveNet model downloaded and saved to 'movenet_saved_model'


In [3]:
# This script is taken directly from your project plan's appendix
import tensorflow as tf
import tensorflow_hub as hub

# Path to the model we just saved in the previous step
saved_model_dir = 'movenet_saved_model'

# Load the hub model again and define a serving default signature
model_url = "https://tfhub.dev/google/movenet/singlepose/lightning/4"
hub_model = hub.load(model_url)

# Define a function that uses the model and add a serving_default signature
@tf.function(input_signature=[tf.TensorSpec(shape=[1, None, None, 3], dtype=tf.uint8)])
def movenet_with_signature(input_image):
    return hub_model.signatures['serving_default'](input_image)

# Save the model with the defined signature
tf.saved_model.save(hub_model, saved_model_dir, signatures={'serving_default': movenet_with_signature})


# 1. Initialize the TFLite Converter
# Specify the signature key 'serving_default' which is the default signature
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir, signature_keys=['serving_default'])

# 2. Apply optimizations (as recommended in the plan)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# 3. Apply float16 quantization (as recommended in the plan)
converter.target_spec.supported_types = [tf.float16]

# 4. Convert the model
tflite_model = converter.convert()

# 5. Save the converted model to a file
tflite_model_path = 'movenet_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Model successfully converted and saved to '{tflite_model_path}'")

INFO:tensorflow:Assets written to: movenet_saved_model\assets


INFO:tensorflow:Assets written to: movenet_saved_model\assets


Model successfully converted and saved to 'movenet_model.tflite'


In [4]:
import os

# Calculate the size of the original SavedModel directory
original_size = sum(os.path.getsize(os.path.join(dirpath, f)) for dirpath, _, filenames in os.walk(saved_model_dir) for f in filenames)

# Get the size of the new TFLite model file
tflite_size = os.path.getsize(tflite_model_path)

print(f"Original Model Size: {original_size / 1024 / 1024:.2f} MB")
print(f"Converted TFLite Model Size: {tflite_size / 1024 / 1024:.2f} MB")
print(f"The TFLite model is significantly smaller, which is perfect for a mobile app!")

Original Model Size: 20.26 MB
Converted TFLite Model Size: 4.59 MB
The TFLite model is significantly smaller, which is perfect for a mobile app!


In [5]:
import cv2
import mediapipe as mp
import numpy as np

# --- 1. Initialize MediaPipe Pose ---
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# --- 2. Setup Video ---
video_input_path = 'C:/Users/sahil/OneDrive/Desktop/xxxx.mp4'
video_output_path = 'output_video_jump_height.mp4'
cap = cv2.VideoCapture(video_input_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_output_path, fourcc, fps, (frame_width, frame_height))

# --- 3. Logic Variables ---
baseline_y = 0
peak_y = 0
is_calibrated = False
calibration_frames = 10 # Use the first 10 frames to calibrate
frame_count = 0
pixels_to_cm_ratio = 0
jump_height_cm = 0

# IMPORTANT: Provide the approximate real-world height of the person in the video
# This is used for the pixel-to-cm conversion
PERSON_HEIGHT_CM = 170.0

print("Starting vertical jump analysis...")

# --- 4. Main Processing Loop ---
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    frame_count += 1

    if results.pose_landmarks:
        try:
            landmarks = results.pose_landmarks.landmark

            # We will track the hip's y-coordinate as the project plan suggests [cite: 99]
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            # Use the average of both hips for stability
            hip_y = (left_hip.y + right_hip.y) / 2.0 * frame_height

            # --- 5. Calibration Phase ---
            if not is_calibrated:
                if frame_count == 1: # First frame setup
                    # Establish the pixel-to-cm ratio from a standing frame
                    left_ankle_y = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * frame_height
                    left_mouth_y = landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].y * frame_height
                    person_height_pixels = abs(left_ankle_y - left_mouth_y)
                    if person_height_pixels > 0:
                        pixels_to_cm_ratio = PERSON_HEIGHT_CM / person_height_pixels

                    # Initialize baseline and peak
                    baseline_y = hip_y
                    peak_y = hip_y

                elif frame_count < calibration_frames:
                    # Refine baseline over a few frames
                    baseline_y = (baseline_y * (frame_count - 1) + hip_y) / frame_count
                    peak_y = baseline_y
                else:
                    is_calibrated = True # Calibration complete
                    print(f"Calibration complete. Baseline Y: {baseline_y:.2f}, Pixel-to-CM Ratio: {pixels_to_cm_ratio:.2f}")

            # --- 6. Tracking Phase ---
            else:
                # In image coordinates, a lower Y value means higher up.
                # So, we look for the minimum Y value to find the peak of the jump.
                if hip_y < peak_y:
                    peak_y = hip_y

        except:
            pass

    # --- 7. Calculate and Render ---
    if is_calibrated:
        # Calculate jump height in pixels
        jump_height_pixels = baseline_y - peak_y

        # Convert pixels to cm
        if pixels_to_cm_ratio > 0:
            jump_height_cm = jump_height_pixels * pixels_to_cm_ratio

        # Draw the baseline and peak lines for visualization
        cv2.line(frame, (0, int(baseline_y)), (frame_width, int(baseline_y)), (0, 255, 0), 2)
        if peak_y < baseline_y:
            cv2.line(frame, (0, int(peak_y)), (frame_width, int(peak_y)), (0, 0, 255), 2)

        # Display the jump height
        cv2.rectangle(frame, (0,0), (400, 70), (0,0,0), -1)
        cv2.putText(frame, f"JUMP HEIGHT: {jump_height_cm:.2f} CM", (15, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    out.write(frame)

# --- 8. Cleanup ---
print(f"Processing complete. Max jump height detected: {jump_height_cm:.2f} cm")
cap.release()
out.release()
pose.close()

Starting vertical jump analysis...
Calibration complete. Baseline Y: 451.44, Pixel-to-CM Ratio: 0.40
Processing complete. Max jump height detected: 50.82 cm
